In [37]:
import pandas as pd
import numpy as np
from scipy.stats import binom_test
from scipy import stats

In [38]:
file_loc = '/Users/gzcheng/Desktop/Udacity/ABTesting/homepage-experiment-data.csv'

In [39]:
df = pd.read_csv(file_loc, index_col='Day')
df.head()

,Control Cookies,Control Downloads,Control Licenses,Experiment Cookies,Experiment Downloads,Experiment Licenses
Day,,,,,,
1,1764,246,1,1850,339,3
2,1541,234,2,1590,281,2
3,1457,240,1,1515,274,1
4,1587,224,1,1541,284,2
5,1606,253,2,1643,292,3


In [40]:
df_ctr, df_exp = pd.DataFrame(), pd.DataFrame()
df_ctr[['cookies','downloads','license']] = df[['Control Cookies','Control Downloads','Control Licenses']]
df_exp[['cookies','downloads','license']] = df[['Experiment Cookies','Experiment Downloads','Experiment Licenses']]

In [41]:
df_ctr['download_ratio'] = df_ctr['downloads']/df_ctr['cookies']
df_exp['download_ratio'] = df_exp['downloads']/df_exp['cookies']
df_ctr['license_ratio'] = df_ctr['license']/df_ctr['cookies']
df_exp['license_ratio'] = df_exp['license']/df_exp['cookies']
df_ctr['type'] = 'ctr'
df_exp['type'] = 'exp'

In [42]:
p_cookies = 3250/2

In [43]:
mu_ctr_cookies, sig_ctr_cookies, n_ctr_cookies = np.mean(df_ctr.cookies), np.std(df_ctr.cookies), df_ctr.shape[0]
mu_exp_cookies, sig_exp_cookies, n_exp_cookies = np.mean(df_exp.cookies), np.std(df_exp.cookies), df_exp.shape[0]

In [44]:
sig_ctr_cookies

114.28536160129036

In [45]:
dtr_ctr = np.random.normal(loc=mu_ctr_cookies, scale=sig_ctr_cookies, size=n_ctr_cookies)
# Perform one-sample t-test
_, pv_ctr_cookies = dtr_ctr.ppf(mu_exp_cookies)
1- pv_ctr_cookies

AttributeError: 'numpy.ndarray' object has no attribute 'ppf'

In [36]:
# dtr_exp = np.random.normal(loc=p_cookies, scale=sig_exp_cookies, size=n_exp_cookies)
# # Perform one-sample t-test
# _, pv_exp_cookies = stats.ttest_1samp(dtr_exp, mu_exp_cookies)
# pv_exp_cookies

0.9440637961334937

In [52]:
n_control = df['Control Cookies'].sum()
n_experiment = df['Experiment Cookies'].sum()
n_obs = n_control + n_experiment

print("Total number of observation: ", n_obs)
print("Total number of control group: ", n_control)
print("Total number of experiment group: ", n_experiment)
# Compute a z-score and p-value
p = 0.5
sd = np.sqrt(p * (1-p) * n_obs)

z = (n_control - p * n_obs) / sd

print("z score: ",z)
print("p value: ", 2 * stats.norm.cdf(z))

Total number of observation:  94197
Total number of control group:  46851
Total number of experiment group:  47346
z score:  -1.6128228329131302
p value:  0.10678300401876312


In [48]:
(n_control + 0.5)

46851.5

In [49]:
- p * n_obs

-47098.5

# check the invariant metrics

In [17]:
df_al = pd.concat([df_ctr,df_exp],axis=0)
df_al.reset_index(drop=True,inplace=True)
df_al

,cookies,downloads,license,download_ratio,license_ratio,type
0,1764,246,1,0.139456,0.000567,null
1,1541,234,2,0.151849,0.001298,null
2,1457,240,1,0.164722,0.000686,null
3,1587,224,1,0.141147,0.000630,null
4,1606,253,2,0.157534,0.001245,null
5,1681,287,3,0.170732,0.001785,null
6,1534,262,5,0.170795,0.003259,null
7,1798,331,12,0.184093,0.006674,null
8,1478,223,30,0.150880,0.020298,null
9,1461,236,32,0.161533,0.021903,null
